In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
# reading dataset 
file = "C:/Users/20201954/Desktop/data/quantitative MRI.xlsx"
df = pd.read_excel(file)

numerical_df = df.select_dtypes(include=[float, int])
non_numerical_df = df.select_dtypes(exclude=[float, int]) 
df = numerical_df

C:\Users\20201954\AppData\Local\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
# Remove rows with any 0 values
df_filtered = df[(df != 0).all(axis=1)]
print(df_filtered)
print(f"Original DataFrame shape: {df.shape}")
print(f"Cleaned DataFrame shape: {df_filtered.shape}")
df = df_filtered

      subjectID  session  sessionID  muscleID  sideID    volume    trk_leng  \
0             1        1          1         1       1  225.5715  128.312424   
1             1        1          1         1       2  217.0665   74.680931   
2             1        1          1         2       1  445.9185   98.776443   
3             1        1          1         2       2  456.3810   87.850719   
4             1        1          1         3       1  641.4390  110.342781   
...         ...      ...        ...       ...     ...       ...         ...   
4735         79        1          1        30       2  266.0040  110.335335   
4736         79        1          1        31       1  164.1870  126.894844   
4737         79        1          1        31       2  164.7675  114.113464   
4738         79        1          1        32       1   54.7290   62.525448   
4739         79        1          1        32       2   68.1210   78.812187   

        trk_ang  trk_seed  trk_dens  ...   dti_snr0

In [4]:
df = df.drop(columns=['subjectID', 'session', 'sessionID', 'sideID', 'volume', 'trk_leng', 'trk_ang'])
df.head()

,muscleID,trk_seed,trk_dens,dti_l1,dti_l2,dti_l3,dti_md,dti_rd,dti_fa,dti_adci,...,dti_snr0,dix_watfr,dix_fatfr,dix_t2star,dix_r2star,dix_snr,dix_dbond,t2_fatfr,t2_t2w,t2_t2f
0,1,0.363608,1.547397,1.755076,1.296643,1.118196,1.385661,1.205056,0.218869,1.381021,...,21.864636,0.988031,1.196883,31.124536,32.360153,72.005936,11.016443,6.893448,29.766680,218.154770
1,1,0.292484,0.862784,1.674851,1.270722,1.088295,1.333043,1.176370,0.219142,1.332976,...,26.351746,0.986770,1.323004,24.200849,41.488840,55.315876,8.083123,6.579949,28.542535,188.994987
2,2,0.361483,1.201093,1.747564,1.318626,1.200244,1.422549,1.259057,0.205252,1.420225,...,24.654427,0.981818,1.818241,28.298574,35.659748,65.425865,10.307537,8.156445,30.253475,211.003571
3,2,0.356667,1.070751,1.727356,1.307701,1.196663,1.412093,1.251865,0.203801,1.413748,...,25.463583,0.983509,1.649060,25.391251,39.659378,65.230667,7.816146,7.977058,29.786484,204.876495
4,3,0.369861,1.960575,1.778001,1.367932,1.214833,1.457258,1.292251,0.195484,1.459163,...,26.109089,0.961433,3.856707,27.586838,36.733351,73.749279,10.102497,10.879514,30.962922,213.111839


In [5]:
# Voor elke spier
#for muscle in df['muscleID'].unique():
    # Filter de data voor de huidige spier
    #muscle_data = df[df['muscleID'] == muscle]
    
    # Voor elke kolom behalve 'muscleID'
   # for col in df.columns[1:]:
    #    plt.figure(figsize=(5, 3))
    #    sns.boxplot(data=muscle_data, x=col)
     #   plt.title(f'Spier {muscle}: {col}')
     #   plt.show()

In [6]:
# Functie om outliers te verwijderen voor een specifieke spier en een specifieke parameterkolom
def remove_outliers(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    filter = (df[col] >= Q1 - 1.5 * IQR) & (df[col] <= Q3 + 1.5 * IQR)
    return df.loc[filter]

# Maak een lege lijst om de gefilterde DataFrames voor elke spier op te slaan
filtered_dfs = []

# Voor elke spier de outliers verwijderen over alle parameters
for muscle in df['muscleID'].unique():
    # Filter de data voor de huidige spier
    muscle_data = df[df['muscleID'] == muscle].copy()
    
    # Voor elke parameter de outliers verwijderen
    for col in df.columns[1:]:
        muscle_data = remove_outliers(muscle_data, col)
    
    # Voeg de gefilterde spier data toe aan de lijst
    filtered_dfs.append(muscle_data)

# Combineer alle gefilterde spier data
df_cleaned = pd.concat(filtered_dfs, ignore_index=True)

# Toon het verschil in grootte tussen het originele en het gefilterde DataFrame
print(f"Original DataFrame shape: {df.shape}")
print(f"Cleaned DataFrame shape: {df_cleaned.shape}")

# Werk de originele DataFrame bij met de gefilterde data
df = df_cleaned

remaining_indices = df.index
# Bekijk de eerste paar rijen van de gefilterde data
print(df.head())


Original DataFrame shape: (4729, 21)
Cleaned DataFrame shape: (3624, 21)
   muscleID  trk_seed  trk_dens    dti_l1    dti_l2    dti_l3    dti_md  \
0         1  0.343121  1.313979  1.919622  1.476775  1.303066  1.569240   
1         1  0.366241  1.115636  1.929991  1.443829  1.299382  1.561221   
2         1  0.373637  1.035097  1.947872  1.522166  1.351802  1.612967   
3         1  0.367740  1.075582  2.014825  1.489711  1.345411  1.621265   
4         1  0.370957  0.955739  2.099751  1.451036  1.264286  1.607739   

     dti_rd    dti_fa  dti_adci  ...   dti_snr0  dix_watfr  dix_fatfr  \
0  1.393511  0.200168  1.570057  ...  23.610182   0.975246   2.475441   
1  1.372815  0.217147  1.561734  ...  37.704342   0.967882   3.211781   
2  1.435240  0.189434  1.615622  ...  30.155842   0.974882   2.511755   
3  1.417856  0.216513  1.626250  ...  36.296944   0.977207   2.279284   
4  1.359291  0.274865  1.607219  ...  27.130775   0.974281   2.571912   

   dix_t2star  dix_r2star     dix_snr

In [7]:
#for muscle in df['muscleID'].unique():
    # Filter de data voor de huidige spier
   # muscle_data = df[df['muscleID'] == muscle]
    
    # Voor elke kolom behalve 'muscleID'
    #for col in df.columns[1:]:
     #   plt.figure(figsize=(5, 3))
      #  sns.boxplot(data=muscle_data, x=col)
      #  plt.title(f'Spier {muscle}: {col}')
      #  plt.show()

In [9]:
# Filter de niet-numerieke kolommen op basis van de overgebleven indices
filtered_non_numerical_df = non_numerical_df.loc[remaining_indices]
# Voeg de dataframes weer samen
final_df = pd.concat([filtered_non_numerical_df, numerical_df ], axis=1)
print(final_df.head())
final_df.info

     subject                 muscle   side  subjectID  session  sessionID  \
0  MOTION001  Lateral_Gastrocnemius   Left          1        1          1   
1  MOTION001  Lateral_Gastrocnemius  Right          1        1          1   
2  MOTION001   Medial_Gastrocnemius   Left          1        1          1   
3  MOTION001   Medial_Gastrocnemius  Right          1        1          1   
4  MOTION001                 Soleus   Left          1        1          1   

   muscleID  sideID    volume    trk_leng  ...   dti_snr0  dix_watfr  \
0         1       1  225.5715  128.312424  ...  21.864636   0.988031   
1         1       2  217.0665   74.680931  ...  26.351746   0.986770   
2         2       1  445.9185   98.776443  ...  24.654427   0.981818   
3         2       2  456.3810   87.850719  ...  25.463583   0.983509   
4         3       1  641.4390  110.342781  ...  26.109089   0.961433   

   dix_fatfr  dix_t2star  dix_r2star    dix_snr  dix_dbond   t2_fatfr  \
0   1.196883   31.124536   32.3

<bound method DataFrame.info of         subject                 muscle   side  subjectID  session  sessionID  \
0     MOTION001  Lateral_Gastrocnemius   Left          1        1          1   
1     MOTION001  Lateral_Gastrocnemius  Right          1        1          1   
2     MOTION001   Medial_Gastrocnemius   Left          1        1          1   
3     MOTION001   Medial_Gastrocnemius  Right          1        1          1   
4     MOTION001                 Soleus   Left          1        1          1   
...         ...                    ...    ...        ...      ...        ...   
4735        NaN                    NaN    NaN         79        1          1   
4736        NaN                    NaN    NaN         79        1          1   
4737        NaN                    NaN    NaN         79        1          1   
4738        NaN                    NaN    NaN         79        1          1   
4739        NaN                    NaN    NaN         79        1          1   

      m

In [10]:
# Controleer de vorm van het resultaat
df = final_df.dropna(inplace = True)
print(final_df.shape)
print(final_df)

(3624, 31)
        subject                     muscle   side  subjectID  session  \
0     MOTION001      Lateral_Gastrocnemius   Left          1        1   
1     MOTION001      Lateral_Gastrocnemius  Right          1        1   
2     MOTION001       Medial_Gastrocnemius   Left          1        1   
3     MOTION001       Medial_Gastrocnemius  Right          1        1   
4     MOTION001                     Soleus   Left          1        1   
...         ...                        ...    ...        ...      ...   
3619  MOTION061           Fibularis_Brevis  Right         61        1   
3620  MOTION061          Tibialis_Anterior   Left         61        1   
3621  MOTION061          Tibialis_Anterior  Right         61        1   
3622  MOTION061  Extensor_Digitorum_Longus   Left         61        1   
3623  MOTION061  Extensor_Digitorum_Longus  Right         61        1   

      sessionID  muscleID  sideID    volume    trk_leng  ...   dti_snr0  \
0             1         1       1  22

In [11]:
# determining the name of the file
file_name = "C:/Users/20201954/Desktop/data/cleaned_qMRI2.xlsx"

# saving the excel
final_df.to_excel(file_name)
print('DataFrame is written to Excel File successfully.')

DataFrame is written to Excel File successfully.


In [12]:
# file with only numerical data and muscle id

final_df1 = final_df.drop(columns=['subject', 'muscle', 'side', 'subjectID', 'session', 'sessionID', 'sideID'])
final_df1.head()

# determining the name of the file
file_name = "C:/Users/20201954/Desktop/data/cleaned_qMRI_numerical2.xlsx"

# saving the excel
final_df1.to_excel(file_name)
print('DataFrame is written to Excel File successfully.')

DataFrame is written to Excel File successfully.
